# Prediction of Bank Loan Default in TensorFlow
## inspired by "A Unique TensorFlow Neural Net Tutorial using a Jupyter Python Notebook" 
### in IBM Data Science Experience
### by John M. Boyer 

Web site: https://www.ibm.com/developerworks/community/blogs/JohnBoyer

GitHub code repo: https://github.com/john-boyer-phd/TensorFlow-Samples

This TensorFlow Neural Network tutorial has several aspects that are unique or not evident in other tutorials like the MNIST handwritten digits tutorial. 

The focus is on business, both in terms of the use case, prediction of bank loan risk, and data and in terms of extra steps needed to help take your data science results to production. 

We’ll cover the following:

* Reading data and reshaping it for TensorFlow neural net input
* Epoch based training and training data randomization
* Training in small batches for larger data sets
* Tuning hyperparameters like the network structure and activation function
* Tricks for properly saving and restoring models for use in a production environment
* How to do transfer learning
* How to derive confidence values for neural net outputs 

#### Note: 

This derived Notebook is a small contribution by Claude Coulombe, PhD candidate, TÉLUQ / UQAM

This is not a deep neural network, in fact it's a pretty shallow one with just an unique hidden layer. It is more an exercise in TensorFlow to show how a simple neural network can be used to predict a loan default using a simple binary classifier. Claude Coulombe, PhD candidate - Téluq / UQAM

## The problem - prediction of loan default

Default is the failure to pay interest or principal on a loan when due. For example when a home buyer fails to make a mortgage payment. 

The goal of this small exercise will be to train and perform inferences with a `TensorFlow` neural network for predicting whether a loan applicant is likely to `default` on a bank loan, based on features of the applicant that may be predictive of their ability to repay a loan. <u>The dependent variable being predicted is the column named ‘default’</u> in the CSV file. The dependent variable is also called the `label` or `target`, and the data in the column is called the `labeled data`. The predictor variables are `age`, `ed` (level of education), `employ` (years with current employer), `address` (years at current address), `income` (household income in thousands), `debtinc` (debt to income ratio x 100), `creddebt` (credit card debt in thousands), and `othdebt` (other debt in thousands). The predictor variables are also called `features`.  

The remaining columns of data are not needed and will be discarded in the code below. When training, the feature values of the instances (rows) of data are fed as input to the neural net, and the weights and biases of the neural network are adjusted so as to minimize `loss`, which coarsely maps to maximizing accuracy of the neural network’s output layer predictions of the labeled data.

## Data

The sample data we’ll be using for training and testing is in a file called `bankLoanData.csv`, which is a sample data file I obtained from my laptop IBM SPSS statistics package. I’ve used this data because I could easily use SPSS to double-check that all the `TensorFlow` code was behaving as I expected. The advantage to both you and me, then, is that we can now easily adapt the resulting `TensorFlow` code to build bigger neural nets that learn from much larger data sets.

The first cell of the Jupyter iPython Notebook below has to do some version of reading the CSV file. In my prior tutorial, I showed how to load a CSV file into a database and then load the data into a Pandas dataframe using a SQL query. For this tutorial, any version of `pandas.read_csv()` will suffice. For example, in IBM Data Science Experience on IBM Cloud, you can simply drag and drop the CSV file to add it as a dataset, and then select “Insert Code” to automatically generate the code to read the CSV file from cloud object storage. For larger datasets, you may prefer to use a `SparkSession Dataframe` instead, but in that case, you’ll need to slightly adjust the `numpy` extraction code in the next Notebook cell.

In [132]:
import pandas as pd

df_data_1 = pd.read_csv('bankloanData.csv')
df_data_1.head()

,age,ed,employ,address,income,debtinc,creddebt,othdebt,default,preddef1,preddef2,preddef3
0,41,3,17,12,176,9.3,11.359392,5.008608,1,0.808394,0.788640,0.213043
1,27,1,10,6,31,17.3,1.362202,4.000798,0,0.198297,0.128445,0.436903
2,40,1,15,14,55,5.5,0.856075,2.168925,0,0.010036,0.002987,0.141023
3,41,1,15,14,120,2.9,2.658720,0.821280,0,0.022138,0.010273,0.104422
4,24,2,2,0,28,17.3,1.787436,3.056564,1,0.781588,0.737885,0.436903


The next cell of code below assumes a Pandas dataframe named ‘df_data_1’ and uses it to extract the data into numpy arrays needed as input to the TensorFlow API. 

In [133]:
df_data_1.dtypes

age           int64
ed            int64
employ        int64
address       int64
income        int64
debtinc     float64
creddebt    float64
othdebt     float64
default      object
preddef1    float64
preddef2    float64
preddef3    float64
dtype: object

In [134]:
df_data_1.values

array([[41, 3, 17, ..., 0.808394327359702, 0.7886404318214371,
        0.21304337612811897],
       [27, 1, 10, ..., 0.19829747615910395, 0.128445387038174,
        0.43690300550604605],
       [40, 1, 15, ..., 0.0100361080990023, 0.00298677834821412,
        0.141022623460993],
       ..., 
       [48, 1, 13, ..., 0.0301374981044824, 0.0325702625943738,
        0.24801041775523303],
       [35, 2, 1, ..., 0.26900345101699397, 0.37854649636973203,
        0.181814378077261],
       [37, 1, 20, ..., 0.006397812918809229, 0.0111731232851226,
        0.30304155578236497]], dtype=object)

The comprehension in the first np.array() removes instances (rows) that have a missing `default` label.

The eight (8) predictor variables are `age`, `ed` (level of education), `employ` (years with current employer), `address` (years at current address), `income` (household income in thousands), `debtinc` (debt to income ratio x 100), `creddebt` (credit card debt in thousands), and `othdebt` (other debt in thousands). The remaining columns of data are not needed and will be discarded in the code below.

In [135]:
import numpy as np

# Make a numpy array from the dataframe, except remove rows with no value for 'default'
i = list(df_data_1.columns.values).index('default')
data = np.array([x for x in df_data_1.values if x[i] in ['0', '1']])

# Remove the columns for preddef1, predef2 and preddef3
data = np.delete(data, slice(9,12), axis=1)

The next cell reshapes the data just a bit. The predictors are separated from the dependent variable. The labeled data is converted from strings to integers, and the dependent array is flattened to one dimension (from one column matrix to a one line matrix) to match the shape of the data that will come from the neural network output layer (one line matrix).  The predictors are converted to all floats to facilitate matrix multiplication with weights and biases within the neural network.

In [136]:
# Separate the 'predictors' (aka 'features') from the dependent variable (aka 'label' or 'target') 
# that we will learn how to predict
predictors = np.delete(data, 8, axis=1)
dependent = np.delete(data, slice(0, 8), axis=1)

# Convert the label (aka 'target' type to numeric categorical 
# representing the classes to predict (binary classifier)
dependent = dependent.astype(int)
print(dependent.shape)
print(dependent[:10],"\n  .\n  .\n  .\n")

# And flatten it to one dimensional for use as the expected output label vector in TensorFlow
dependent = dependent.flatten()
print(dependent.shape)
print(dependent[:10],"...")

# Convert all the predictors to float to simplify this demo TensorFlow code
predictors = predictors.astype(float)

# Get the shape of the predictors
m, n = predictors.shape
m, n

(700, 1)
[[1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]] 
  .
  .
  .

(700,)
[1 0 0 0 1 0 0 0 1 0] ...


(700, 8)

In [137]:
predictors[0]

array([  41.      ,    3.      ,   17.      ,   12.      ,  176.      ,
          9.3     ,   11.359392,    5.008608])

In [138]:
dependent

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0,
       0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 1,

The next cell simply takes the first 500 instances as training data, leaving the remaining 200 instances for a test set. It’s not unusual to randomly select the training and test sets from the given data, but this particular sample was already random.  It’s also typical to choose about a 70/30 percent split for training and test, and this code does so, except for rounding to a size divisible by the training batch size we’ll define later (700*0.7 => 489.999 rounding => 500). 

This cell also defines a method that returns batch-sized slices of the training data. If the training data were too large to fit in memory, then this method could instead load data one batch at a time, such as with a SQL query.

In [139]:
# Partition the input data into a training set and a test set

m_train = 500
m_test = m - m_train

predictors_train = predictors[:m_train]
dependent_train = dependent[:m_train]

predictors_test = predictors[m_train:]
dependent_test = dependent[m_train:]

# Gets a batch of the training data. 
# NOTE: Rather than loading a whole large data set as above and then taking array slices as done here, 
#       This method can connect to a data source and select just the batch needed.
def get_training_batch(batch_num, batch_size):
    lower = batch_num * (m_train // batch_size)
    upper = lower + batch_size
    return predictors_train[lower:upper], dependent_train[lower:upper]

## Set-up the neural network using TensorFlow

Now we’re set to start with some actual TensorFlow code. This next cell imports TensorFlow, makes a few useful initializations, and then defines a method that will build a neural network layer of a given size, fully connect it to a preceding layer, and set its output activation function.

In [140]:
import tensorflow as tf

# Make this notebook's output stable across runs 
# ensure reproducibility of the results
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

# A method to build a new neural net layer of a given size,  
# fully connect it to a given preceding layer X, and 
# compute its output Z either with or without (default) an activation function
# Call with activation=tf.nn.relu or tf.nn.sigmoid or tf.nn.tanh, for examples

def make_nn_layer(layer_name, layer_size, X, activation=None):
    with tf.name_scope(layer_name):
        X_size = int(X.get_shape()[1])
        SD = 2 / np.sqrt(X_size)
        weights = tf.truncated_normal((X_size, layer_size), dtype=tf.float64, stddev=SD)
        W = tf.Variable(weights, name='weights')
        b = tf.Variable(tf.zeros([layer_size], dtype=tf.float64), name='biases')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

Now we can add the code cell that builds the neural network structure. In this case, we’re going to have one input layer (X), one hidden layer (hidden1), and one output layer (outputs). 

The ### n_hidden2 = n // 2 comments show how to add more hidden layers, but with this sample data, we’re going to be able to learn everything we can with only one layer. The output layer has two nodes, one for outputting class 0 (the loan applicant won’t default) and the other for class 1 (the loan applicant will default). The ‘y’ variable will be used during training to store the labeled data we expect to match with the output layer.

One line of code that helps makes this tutorial unique is the one that creates a tf.identity() node that gives the name ‘nn_output’. This enables us to save a name for the output layer so that we can recover and use the output layer after a restore.

In [141]:
# Make the neural net structure

n_inputs = n
n_hidden1 = n 
### n_hidden2 = n // 2
n_outputs = 2   # Two output classes: defaulting or non-defaulting on loan

X = tf.placeholder(tf.float64, shape=(None, n_inputs), name='X')

with tf.name_scope('nn'):
    hidden1 = make_nn_layer('hidden1', n_hidden1, X, activation=tf.nn.relu)
    hidden2 = hidden1
    ### hidden2 = make_nn_layer('hidden2', n_hidden2, hidden1, activation=tf.nn.relu)
    outputs = make_nn_layer('outputs', n_outputs, hidden2) 
    outputs = tf.identity(outputs, "nn_output")
    
y = tf.placeholder(tf.int64, shape=(None), name='y')

The cell above and the next cell are where most of the hyperparameter tuning occurs. Neural network is just the algorithm. The input parameters passed to a neural network during inference are the feature values. During training, the input parameters are feature values and the expected output labeled data. But the neural network is adaptable beyond those input parameters, and so these configurable parts are called hyperparameters. The number and size of the hidden layers are among the hyperparameters, as is the activation function.  For examples, you can try other numbers and sizes of hidden layers, and ‘tanh’ and ‘sigmoid’ are other activation functions to try. However, the given configuration seems to do very near the best on this data.

What we’ve done so far is to create the main part of a TensorFlow compute graph that happens to have the shape needed for a neural network. 

What we’re going to do in the next cell below is attach two different root nodes to the output layer, one that adds functionality for training and the other for testing. The `training_op` uses the gradient descent method for minimizing loss (of perfect confidence in the correct answers and zero confidence in incorrect answers, where the correct answers are provided by the labeled data that will be in `y`).

In [142]:
# Define how the neural net will learn

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=outputs)
    loss = tf.reduce_mean(xentropy, name='l')
    
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("test"):
    correct = tf.nn.in_top_k(tf.cast(outputs, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    

Now we’re going to do a quick little cell that sets up our ability to save the model once it is trained. 

In [143]:
# Set up the ability to save and restore the trained neural net...

init = tf.global_variables_initializer()
saver = tf.train.Saver()

You only need to do these mkdir commands the first time you run the notebook, so you may want to put them in a separate cell to make it easier to skip them. Also, in Data Science Experience Local, you only need the second mkdir.

In [144]:
# ... and make a subdirectory space in which to save the model files (only need to run this once)
!mkdir "../datasets"
!mkdir "../datasets/Neural Net"

mkdir: ../datasets: File exists
mkdir: ../datasets/Neural Net: File exists


## Training models

Now we can have the magic notebook cell that trains and saves the trained model. Each epoch of training exposes the neural net to the entire set of training data. When you see this code run, you will see accuracy increase over the many epochs, just as biological neural networks learn through repetition. For each epoch, we run through the training data in batches, to simulate how we’d handle a larger training set. Each batch of features and corresponding labeled data is fed to the `training_op` root node in the compute graph, which is run by `training_session.run()`.

One aspect of this tutorial that is made evident (relative to other tutorials) is the randomization of the training data that takes place at the beginning of each epoch.  This essentially drives different data into the batches in each epoch, which dramatically improves accuracy over a larger number of epochs (though it is much easier programmatically to do this randomization when all data fits into memory).

> Yet another reason why this tutorial is unique is that we’ll actually take a little sidebar to understand why, when doing business with a real stakeholder customer, we need to have a second test set, often called a <u>validation set</u> or a blind set. Why do we need a second test set? When I ask this, the usual reply is something like, “I don’t know, to double-check accuracy?”  Well, sort of. But if you look at the structure of training, the weights and biases are affected not just by the training data. Indirectly, they are also affected by the test data because we choose `n_epochs` to keep running training epochs until we get the best accuracy on the test set. In other words, we’re teaching to the test. The validation set or blind set has no such indirect effect on the weights and biases computed for the neural network. It is simply another test set that, to ensure construct validity, should be randomly from the same pool of data that the training set and test set are randomly selected from. In this way, the validation set is not just the `final exam`, it’s the first experience of the real world. Sidebar complete.

Once all training has been done, we save the trained model into the previously created datasets subdirectory. In this sample code, we are only saving at the end, but this same command can be used to save the intermediate results of a very long training run.

In [145]:
# TRAINING TIME

# This is how many times to use the full set of training data
n_epochs = 3000

# For a larger training set, it's typically necessary to break training into
# batches so only the memory needed to store one batch of training data is used
batch_size = 50

with tf.Session() as training_session:
    init.run()
    
    for epoch in range(n_epochs):
        
        # Shuffling (across batches) is easier to do for small data sets and
        # helps increase accuracy
        training_set = [[pt_elem, dependent_train[i]] for i, pt_elem in enumerate(predictors_train)]
        np.random.shuffle(training_set)
        predictors_train = [ts_elem[0] for ts_elem in training_set]
        dependent_train = [ts_elem[1] for ts_elem in training_set]
       
        # Loop through the whole training set in batches
        for batch_num in range(m_train // batch_size):
            X_batch, y_batch = get_training_batch(batch_num, batch_size)
            training_session.run(training_op, feed_dict={X: X_batch, y: y_batch})

        if epoch % 100 == 99:
            acc_train = accuracy.eval(feed_dict={X: predictors_train, y: dependent_train})
            acc_test = accuracy.eval(feed_dict={X: predictors_test, y: dependent_test})
            print(epoch+1, "Training accuracy:", acc_train, "Testing accuracy:", acc_test)

    save_path = saver.save(training_session, "../datasets/Neural Net/Neural Net.ckpt")
    
    # A quick test with the trained model 
    Z = outputs.eval(feed_dict={X: predictors_test[:20]})
    dependent_pred = np.argmax(Z, axis=1)
    print("")
    print("Actual classes:   ", dependent_test[:20])  
    print("Predicted classes:", dependent_pred)

100 Training accuracy: 0.76 Testing accuracy: 0.755
200 Training accuracy: 0.774 Testing accuracy: 0.79
300 Training accuracy: 0.798 Testing accuracy: 0.82
400 Training accuracy: 0.798 Testing accuracy: 0.835
500 Training accuracy: 0.808 Testing accuracy: 0.825
600 Training accuracy: 0.796 Testing accuracy: 0.815
700 Training accuracy: 0.802 Testing accuracy: 0.81
800 Training accuracy: 0.788 Testing accuracy: 0.815
900 Training accuracy: 0.81 Testing accuracy: 0.815
1000 Training accuracy: 0.82 Testing accuracy: 0.81
1100 Training accuracy: 0.818 Testing accuracy: 0.81
1200 Training accuracy: 0.806 Testing accuracy: 0.805
1300 Training accuracy: 0.802 Testing accuracy: 0.815
1400 Training accuracy: 0.812 Testing accuracy: 0.82
1500 Training accuracy: 0.806 Testing accuracy: 0.8
1600 Training accuracy: 0.816 Testing accuracy: 0.81
1700 Training accuracy: 0.806 Testing accuracy: 0.81
1800 Training accuracy: 0.822 Testing accuracy: 0.825
1900 Training accuracy: 0.802 Testing accuracy: 0.

The data files that TensorFlow created during the save operation can be transported to a production environment. The neural network can then be restored using the code in the next cell below, and the output layer can be obtained and used for inference (using `get_tensor_by_name()`).  In fact, showing how to do that is part of what makes this tutorial unique, as even the current TensorFlow documentation for save/restore (incorrectly) reuses variables after restore that were defined before save (rather than running variables obtained from the restored graph). The code below also shows how to reference into the hierarchy of a namescope. 

> As another sidebar unique in this tutorial, note that you can also use this method of naming with `tf.identity` and then getting the tensor from a restored graph to do transfer learning between neural nets. Specifically, once you create a hidden layer with `make_nn_layer()`, you can name it with `tf.identity`. Then, you train and save as shown above. Then, to transfer to a second neural net, you restore the trained and saved one, get the hidden layer by name, attach alternate fully connected hidden layers as needed, and an alternate output layer, and then train the new second neural network using the methodology above. Sidebar complete.

The cell below mocks up having a REST API receiving a batch of feature instances and converting them to a numpy array by simply taking a slice of the test data.  With the inference TensorFlow session, the compute graph and the values it contained are restored. After that, we obtain the tensor corresponding to the neural network output layer by using the name we previously assigned.  Then, we run the output layer, giving the batch of feature instances to the input layer ‘X’ (`inference_session.run()`). The predictions of the dependent variable are then obtained by choosing whichever of the two output layer nodes has the higher value (using `np.argmax()`).

Finally, to add one more feature that makes this tutorial unique, let’s look at how to get the actual confidence values for the predictions. Somehow, this may not have seemed as important when doing the MNIST hand-written digit tutorial, but in a business context it’s important to know how much confidence we have in an answer. 

To get the confidence values, we do something interesting with the compute graph. Remember, it’s just a compute graph and won’t bite. In this case, we pop a new root node onto the output layer to apply the softmax function, which gives the probability of occurrence of each output value. Then, we do one last comprehension to ferret out the confidences of the predicted labels for each feature instance.

In [147]:
# Restore the saved model and use it to perform inference on a "received" new set of data

# We will simulate "receiving" the new data by taking a slice of the test set.
predictors_received = predictors_test[20:40]

import tensorflow as tf_inference

with tf_inference.Session() as inference_session:
    inf_saver = tf_inference.train.import_meta_graph('../datasets/Neural Net/Neural Net.ckpt.meta')
    inf_saver.restore(inference_session, tf_inference.train.latest_checkpoint('../datasets/Neural Net/'))
    
    graph = tf_inference.get_default_graph()    
    nn_output = graph.get_tensor_by_name("nn/nn_output:0")

    Z = inference_session.run(nn_output, feed_dict={X: predictors_received})
    dependent_pred = np.argmax(Z, axis=1)
    
    dependent_prob = inference_session.run(tf_inference.nn.softmax(nn_output), feed_dict={X: predictors_received})

    confidences = [p[dependent_pred[i]] for i, p in enumerate(dependent_prob)]
    
print()
print("Actual classes:   ", dependent_test[20:40])
print("Predicted classes:", dependent_pred)
print("")
print("Confidences:\n", confidences)
print("")
print("Probabilities:\n", dependent_prob)

INFO:tensorflow:Restoring parameters from ../datasets/Neural Net/Neural Net.ckpt

Actual classes:    [0 0 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 1 0 1]
Predicted classes: [0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1]

Confidences:
 [0.72115788841227302, 0.80607047578305713, 0.83934041973914708, 0.99522251895376912, 0.63583707224033514, 0.92301370061024191, 0.9246727235093638, 0.78439759502501416, 0.99999999999583422, 0.64561868763996888, 0.72736408931601459, 0.64561868763996888, 0.83959328566375235, 0.94291801514712859, 0.68170960394665236, 0.99981345698850999, 0.96778595641239296, 0.59248611260528017, 0.99293831799558985, 0.52204835458766652]

Probabilities:
 [[  7.21157888e-01   2.78842112e-01]
 [  8.06070476e-01   1.93929524e-01]
 [  8.39340420e-01   1.60659580e-01]
 [  9.95222519e-01   4.77748105e-03]
 [  6.35837072e-01   3.64162928e-01]
 [  9.23013701e-01   7.69862994e-02]
 [  9.24672724e-01   7.53272765e-02]
 [  7.84397595e-01   2.15602405e-01]
 [  1.00000000e+00   4.16586334e-12]
 [  3.54381

In [90]:
# For when you want to wipe out the training and do it again
!rm -rf "../datasets/Neural Net"

In [91]:
!ls -l "../datasets/Neural Net"

ls: ../datasets/Neural Net: No such file or directory


In [92]:
!ls -l ".."

total 32
-rw-r--r--  1 claudecoulombe  staff  11357 11 nov 03:06 LICENSE
drwxr-xr-x  5 claudecoulombe  staff    170 12 nov 01:10 Neural Net
-rw-r--r--  1 claudecoulombe  staff    252 11 nov 03:06 README.md
drwxr-xr-x  2 claudecoulombe  staff     68 12 nov 01:12 datasets


In [93]:
!rm -rf "../datasets"

And that’s a wrap. Now, it’s your turn. Go ahead, get started with that free IBM Data Science Experience account so you can amaze your friends and bosses with your newfound TensorFlow AI machine learning data science hyperparametric hyperwizardry. You know you waaaanna!